In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import from_numpy
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
torch.manual_seed(1) #랜덤 시드 고정


In [3]:
#Data Load and preprocessing
# Dataset 상속
class CustomDataset(Dataset): 
  def __init__(self, xd, yd):
    self.x_data = xd
    self.y_data = yd

  # 총 데이터의 개수를 리턴
  def __len__(self): 
    return len(self.x_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
  def __getitem__(self, idx): 
    x = self.x_data[idx]
    y = self.y_data[idx]
    return x, y

data = pd.read_csv('/content/drive/MyDrive/Project/NLPModel/Data/EmbeddingTrainData2.csv')
data = data.to_numpy()
data = np.delete(data, 0, axis=0)
data = np.delete(data, 0, axis=1)

train_data = data[:18071]
test_data = data[18072:]

train_x = []
train_y = []
test_x = []
test_y = []

for line in train_data:
  train_x.append(line[:27])
  train_y.append(line[28])

for line in test_data:
  test_x.append(line[:27])
  test_y.append(line[28])



outer_temp_train = []
for line in train_x:
  temp = []
  for word in line:
    word = word.replace('[', '')
    word = word.replace(']', '')
    temp.append(list(map(float, word.split(','))))
  
  outer_temp_train.append(temp)

train_x = outer_temp_train

outer_temp_test = []
for line in test_x:
  temp = []
  for word in line:
    word = word.replace('[', '')
    word = word.replace(']', '')
    temp.append(list(map(float, word.split(','))))
  
  outer_temp_test.append(temp)

test_x = outer_temp_test

train_x = torch.tensor(train_x)
train_y = torch.tensor(train_y)
test_x = torch.tensor(test_x)
test_y = torch.tensor(test_y)

dataset = CustomDataset(train_x, train_y)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)



In [6]:
print(train_y)

tensor([8, 8, 8,  ..., 9, 9, 9])


In [ ]:
# print(train_x.shape)
# print(len(train_x))
# print(train_y.shape)
# print(len(train_y))

print(train_x[0])

tensor([[-0.9492, -0.4853, -0.1203],
        [ 2.1097, -1.8225, -3.9387],
        [ 0.3739, -0.3637, -1.2276],
        [ 0.9225,  0.7259, -2.5336],
        [ 1.3480, -0.4827, -3.1020],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000]])


In [9]:
model = nn.Conv2d(
    in_channels = 1,
    out_channels = 1,
    kernel_size = 1,
    groups = 1,
    stride = 1
)
# model = nn.Linear(27, 8)
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 500
for epoch in tqdm(range(nb_epochs + 1)):
  for batch_idx, samples in enumerate(dataloader):
    x_train, y_train = samples
    # H(x) 계산
    hypothesis = model(x_train)
    # cost 계산
    cost = F.cross_entropy(hypothesis, y_train.unsqueeze(0))
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    # 20번마다 로그 출력
    # if epoch % 100 == 0:
    #   prediction = hypothesis >= torch.FloatTensor([0.5]) # 예측값이 0.5를 넘으면 True로 간주
    #   correct_prediction = prediction.float() == y_train # 실제값과 일치하는 경우만 True로 간주
    #   accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산
    #   print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format( # 각 에포크마다 정확도를 출력
    #     epoch, nb_epochs, cost.item(), accuracy * 100,
    #   ))

100%|██████████| 501/501 [46:59<00:00,  5.63s/it]


In [10]:
torch.save(model, '/content/drive/MyDrive/Project/NLPModel/Data/SoftmaxModel.pt')